<a href="https://colab.research.google.com/github/Sallb679/SafeGraph-Analysis/blob/main/Econ433_proj1_Boubacar_Sall_Chili's_Grill_%26_Bar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The code below is to connect my file directory to my notebook so I can access the data















In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


I am using these libraries because they have the functions I need to manipulate the data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from tabulate import tabulate

Question 1a (1 point): Before you zoom into your restaurant chain, how many observations do you have in total in all datasets? What is the unit of observation in the data? (Note: you do not have to -- and please don't -- concatenate the full datasets together to answer this question. You should iterate over the five datasets and record the number of observations in each of the five data before adding them up)

Hint: Python allows you to read in a zipped data file directly, there is no need to unzip the data beforehand, for example datax=pd.read_csv(filename, compression='zip') will read a raw dataset called filename into a python data called datax.

In [ ]:
# Initialize a counter for total observations
total_observations = 0

# Loop through the years from 2018 to 2022
for year in range(2018, 2023):
    # Generate the file path using f-string
    file_path = f'/content/drive/MyDrive/ECON_433_Project_1_data/weekly_patterns_{year}_sample.csv.zip'

    # Load the dataset
    data = pd.read_csv(file_path, compression='zip')

    # Add the number of rows (observations) in the current dataset to the total count
    total_observations += len(data)

    #Print the number of observations for the current year
    print(f"Year {year}: {len(data)} observations")

# Print the total number of observations from all datasets
print(f"Total observations from 2018-2022: {total_observations}")




Year 2018: 1296498 observations
Year 2019: 1275056 observations
Year 2020: 1280931 observations
Year 2021: 1294674 observations
Year 2022: 1315076 observations
Total observations from 2018-2022: 6462235


The unit of observation is a specific resturant over the course of one week. The output displays the observations for each year and the total over the 5 years.

Question 1b (1 point): First condition on your restaurant chain for each of the datasets, and then concatenate them together. How many observations do you have now?


(Hint: for 1a and 1b, you can either read in each patterns data one by one, subset the data for your restaurant chain and concat them, or you can try to complete the sample code I put at the end of data_notes_2, which does everything in a for loop.)

In [ ]:
# Define the name of the restaurant chain
restaurant_chain_name = "Chili's Grill & Bar"

# Initialize an empty list to store the filtered Data
filtered_data = []

# Loop through the years from 2018 to 2022
for year in range(2018, 2023):
    # Generate the file path for the current year using an f-string
    file_path = f'/content/drive/MyDrive/ECON_433_Project_1_data/weekly_patterns_{year}_sample.csv.zip'

    # Load the dataset
    data = pd.read_csv(file_path, compression='zip')

    # Filter rows where the restaurant chain matches
    filtered_df = data[data['brands'] == restaurant_chain_name]

    # Append the filtered DataFrame to the list
    filtered_data.append(filtered_df)

# Concatenate all the filtered DataFrames into one DataFrame
concatenated_data = pd.concat(filtered_data)

# Print the total number of observations
total_observations = len(concatenated_data)
print(f"Total number of observations for {restaurant_chain_name}: {total_observations}")


Total number of observations for Chili's Grill & Bar: 56242


The output above shows there are 56242 "place-weeks" for Chili's grill and bar across the 5 datasets they are now conctenated together.

Question 1c (1 point):  From now on, I am going to call the concatenated data of your restaurant chain only as your “wide sample”. What is the data type of each variable in your wide sample? How many unique places do you observe for your chain in the US? Is it consistent with the number of units your chain should have if you search for this number online? Comment on your findings. Try to come up with at least one explanation if the two numbers are different.



Now, save your wide data to a .csv file.

In [ ]:
# Check data types of each column in the concatenated DataFrame
print(concatenated_data.dtypes)


# Count unique places using 'placekey'
unique_places = concatenated_data['placekey'].nunique()
print(f"Number of unique places: {unique_places}")

# Save the wide sample to a CSV file
concatenated_data.to_csv('/content/drive/MyDrive/ECON_433_Project_1_data/chilis_wide_sample.csv', index=False)


placekey               object
city                   object
region                 object
date_range_start       object
date_range_end         object
raw_visit_counts        int64
visits_by_day          object
safegraph_brand_ids    object
naics_code              int64
postal_code             int64
brands                 object
dtype: object
Number of unique places: 232


Online I found there are 1222 chili's grill and bar in the US compared to the data set with 232 there may be a difference because SafeGraph may not have data for all locations, especially if some locations don't have enough foot traffic or don't meet certain inclusion criteria. The output above also shows the datatypes for each variable(column).

Question 2a (1 point): In the raw data, the date variables are string variables. Convert them into python’s datetime format, in unit "day". What are the beginning date and ending date in your “wide sample”?

In [ ]:
# Remove timezone information directly from 'date_range_start' and 'date_range_end' columns
concatenated_data['date_range_start'] = concatenated_data['date_range_start'].str.replace(r'([+-]\d{2}:\d{2})', '', regex=True)
concatenated_data['date_range_end'] = concatenated_data['date_range_end'].str.replace(r'([+-]\d{2}:\d{2})', '', regex=True)

# Convert the cleaned date strings back to datetime format
concatenated_data['date_range_start'] = pd.to_datetime(concatenated_data['date_range_start'], errors='coerce')
concatenated_data['date_range_end'] = pd.to_datetime(concatenated_data['date_range_end'], errors='coerce')

# Check for invalid dates
invalid_dates_start = concatenated_data['date_range_start'].isna().sum()
invalid_dates_end = concatenated_data['date_range_end'].isna().sum()

# Print out how many invalid dates were found
print(f"Invalid 'date_range_start' entries: {invalid_dates_start}")
print(f"Invalid 'date_range_end' entries: {invalid_dates_end}")

# Find the earliest and latest valid dates
beginning_date = concatenated_data['date_range_start'].min()
ending_date = concatenated_data['date_range_end'].max()

# Print the results
print(f"Beginning date in the wide sample: {beginning_date}")
print(f"Ending date in the wide sample: {ending_date}")




Invalid 'date_range_start' entries: 0
Invalid 'date_range_end' entries: 0
Beginning date in the wide sample: 2018-01-01 00:00:00
Ending date in the wide sample: 2023-01-02 00:00:00


The output above confirms all of the dates were transformed and finds the beginning and ending date in the sample. Below is an output of the first 5 observations and all the dtypes showing that the date range was transformed.

In [ ]:
concatenated_data.head(5)

,placekey,city,region,date_range_start,date_range_end,raw_visit_counts,visits_by_day,safegraph_brand_ids,naics_code,postal_code,brands
0,222-222@5pr-4pg-9fz,Lees Summit,MO,2018-12-24,2018-12-31,261,"[26,1,43,45,52,53,41]",SG_BRAND_072efa4069c7217d82229845c60e6a0d,722511,64081,Chili's Grill & Bar
1,223-222@8f2-sqt-kj9,Mount Pleasant,TX,2018-08-27,2018-09-03,138,"[10,24,14,13,23,24,30]",SG_BRAND_072efa4069c7217d82229845c60e6a0d,722511,75455,Chili's Grill & Bar
2,223-222@5pv-mqc-t5f,Peoria,IL,2018-03-19,2018-03-26,111,"[10,10,16,15,21,16,23]",SG_BRAND_072efa4069c7217d82229845c60e6a0d,722511,61614,Chili's Grill & Bar
3,zzw-222@62j-wsq-ysq,Dover,NH,2018-05-07,2018-05-14,113,"[13,15,18,9,26,19,13]",SG_BRAND_072efa4069c7217d82229845c60e6a0d,722511,3820,Chili's Grill & Bar
4,228-222@5qv-x6c-gzf,Euless,TX,2018-02-12,2018-02-19,175,"[26,14,34,9,34,34,24]",SG_BRAND_072efa4069c7217d82229845c60e6a0d,722511,76039,Chili's Grill & Bar


In [ ]:
#print the data types
print(concatenated_data.dtypes)

placekey                       object
city                           object
region                         object
date_range_start       datetime64[ns]
date_range_end         datetime64[ns]
raw_visit_counts                int64
visits_by_day                  object
safegraph_brand_ids            object
naics_code                      int64
postal_code                     int64
brands                         object
dtype: object


Question 2b (1 point): In the raw data, the variable visits_by_day is text only. It contains a string of list, consists of multiple integers separated by comma (e.g. “[3,2,4,0,5,2,1]”). Please split the variable into new variables (dailyvisits1, …, dailyvisits7), each of which is an integer. How many variables do you have in the “wide sample” now?

Hint: you may need to create 7 new columns to store the split data from visits_by_day.

In [ ]:
# Remove brackets and split the string into a list of integers
concatenated_data['visits_by_day'] = concatenated_data['visits_by_day'].str.strip("[]")  # Remove brackets
daily_visits = concatenated_data['visits_by_day'].str.split(',', expand=True).astype(int)  # Split and convert to int

# Assign to new columns
concatenated_data[['dailyvisits1', 'dailyvisits2', 'dailyvisits3', 'dailyvisits4', 'dailyvisits5', 'dailyvisits6', 'dailyvisits7']] = daily_visits

# Check the number of columns in the wide sample
num_columns = concatenated_data.shape[1]
print(f"Total number of variables in the wide sample now: {num_columns}")


Total number of variables in the wide sample now: 18


I created new column/variables to tacke the daily visits and they are outputted below to see they are correct.

In [ ]:
concatenated_data.head()

,placekey,city,region,date_range_start,date_range_end,raw_visit_counts,visits_by_day,safegraph_brand_ids,naics_code,postal_code,brands,dailyvisits1,dailyvisits2,dailyvisits3,dailyvisits4,dailyvisits5,dailyvisits6,dailyvisits7
0,222-222@5pr-4pg-9fz,Lees Summit,MO,2018-12-24,2018-12-31,261,"26,1,43,45,52,53,41",SG_BRAND_072efa4069c7217d82229845c60e6a0d,722511,64081,Chili's Grill & Bar,26,1,43,45,52,53,41
1,223-222@8f2-sqt-kj9,Mount Pleasant,TX,2018-08-27,2018-09-03,138,"10,24,14,13,23,24,30",SG_BRAND_072efa4069c7217d82229845c60e6a0d,722511,75455,Chili's Grill & Bar,10,24,14,13,23,24,30
2,223-222@5pv-mqc-t5f,Peoria,IL,2018-03-19,2018-03-26,111,"10,10,16,15,21,16,23",SG_BRAND_072efa4069c7217d82229845c60e6a0d,722511,61614,Chili's Grill & Bar,10,10,16,15,21,16,23
3,zzw-222@62j-wsq-ysq,Dover,NH,2018-05-07,2018-05-14,113,"13,15,18,9,26,19,13",SG_BRAND_072efa4069c7217d82229845c60e6a0d,722511,3820,Chili's Grill & Bar,13,15,18,9,26,19,13
4,228-222@5qv-x6c-gzf,Euless,TX,2018-02-12,2018-02-19,175,"26,14,34,9,34,34,24",SG_BRAND_072efa4069c7217d82229845c60e6a0d,722511,76039,Chili's Grill & Bar,26,14,34,9,34,34,24


Question 2c (2 points): Reshape your “wide sample” so that each observation is a place-date instead of a place-week. Now all the 7 variables (dailyvisits1, …, dailyvisits7) should consolidate into one variable (dailyvisits). I will call the reshaped data your “long sample”. How many observations do you have in the “long sample”? Please comment on the pros and cons of the “long sample” relative to the “wide sample”.

Convert the dailyvisits variable into an integer variable directly. Does it work? If not, explain why and find a way to address the problem. How many observations do you have now after you resolve the issue?

In [ ]:
# Step 1: Reshape the DataFrame from wide to long format
long_sample = pd.melt(concatenated_data,
                      id_vars=['placekey', 'city', 'region', 'date_range_start', 'date_range_end',
                                'raw_visit_counts', 'visits_by_day', 'safegraph_brand_ids',
                                'naics_code', 'postal_code', 'brands'],
                      value_vars=['dailyvisits1', 'dailyvisits2', 'dailyvisits3',
                                  'dailyvisits4', 'dailyvisits5', 'dailyvisits6', 'dailyvisits7'],
                      var_name='day',
                      value_name='dailyvisits')

# Step 2: Convert the 'dailyvisits' column to integer
long_sample['dailyvisits'] = long_sample['dailyvisits'].astype(int)

# Step 3: Final count of observations after resolving issues
num_observations_final = long_sample.shape[0]
print(f"Number of observations: {num_observations_final}")


Number of observations: 393694


After transforming the wide sample to long there are 393,694 observation which is 7 times 56,242 which makes sense as each place-week is now 7 place-dates.

Pros and Cons of the Long Sample vs. Wide Sample
Pros:

Simplicity- Easier to handle in statistical analyses and visualizations, especially for time-series data.
Flexibility- Facilitates aggregation, filtering, and operations over time.

Cons:
Increased Size- The number of rows can increase significantly, which may lead to performance issues with large datasets.

Below is the outputted data showing the shape of the new dataset with more column variables.

In [ ]:
long_sample.head()


,placekey,city,region,date_range_start,date_range_end,raw_visit_counts,visits_by_day,safegraph_brand_ids,naics_code,postal_code,brands,day,dailyvisits
0,222-222@5pr-4pg-9fz,Lees Summit,MO,2018-12-24,2018-12-31,261,"26,1,43,45,52,53,41",SG_BRAND_072efa4069c7217d82229845c60e6a0d,722511,64081,Chili's Grill & Bar,dailyvisits1,26
1,223-222@8f2-sqt-kj9,Mount Pleasant,TX,2018-08-27,2018-09-03,138,"10,24,14,13,23,24,30",SG_BRAND_072efa4069c7217d82229845c60e6a0d,722511,75455,Chili's Grill & Bar,dailyvisits1,10
2,223-222@5pv-mqc-t5f,Peoria,IL,2018-03-19,2018-03-26,111,"10,10,16,15,21,16,23",SG_BRAND_072efa4069c7217d82229845c60e6a0d,722511,61614,Chili's Grill & Bar,dailyvisits1,10
3,zzw-222@62j-wsq-ysq,Dover,NH,2018-05-07,2018-05-14,113,"13,15,18,9,26,19,13",SG_BRAND_072efa4069c7217d82229845c60e6a0d,722511,3820,Chili's Grill & Bar,dailyvisits1,13
4,228-222@5qv-x6c-gzf,Euless,TX,2018-02-12,2018-02-19,175,"26,14,34,9,34,34,24",SG_BRAND_072efa4069c7217d82229845c60e6a0d,722511,76039,Chili's Grill & Bar,dailyvisits1,26


Question 2d (2 points): In your “long sample”, describe the variable dailyvisits. What are the mean, standard deviation, minimum, and maximum of this variable? Comment on the presence and extent of outliers. You can use whatever definition of “outlier”, as long as you state the definition clearly.

Also comment on missing values --- to what extent do you observe missing values in dailyvisits? What could be the economic reasons for these missing values? How can you use your data to tell whether these reasons are relevant for your data?

How would outliers and/or missing values affect the quality of the dataset when you, as a business manager of the chain, attempt to make business decisions based on this dataset? Name at least one decisions that might be immune to these data issues and another one that might be very sensitive to these data issues.


In [ ]:
# Step 1: Descriptive statistics for dailyvisits
mean_dailyvisits = long_sample['dailyvisits'].mean()
std_dailyvisits = long_sample['dailyvisits'].std()
min_dailyvisits = long_sample['dailyvisits'].min()
max_dailyvisits = long_sample['dailyvisits'].max()

print(f"Mean: {mean_dailyvisits}")
print(f"Standard Deviation: {std_dailyvisits}")
print(f"Minimum: {min_dailyvisits}")
print(f"Maximum: {max_dailyvisits}")


Mean: 26.854059243981368
Standard Deviation: 29.779665803537505
Minimum: 0
Maximum: 819


The output above shows the descriptive stats for the dataset including mean, stdev, min, and max.

In [ ]:
# Step 2: Identify outliers
Q1 = long_sample['dailyvisits'].quantile(0.25)
Q3 = long_sample['dailyvisits'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers = long_sample[(long_sample['dailyvisits'] < lower_bound) | (long_sample['dailyvisits'] > upper_bound)]
num_outliers = outliers.shape[0]

print(f"Number of outliers: {num_outliers}")

Number of outliers: 15056


I am using 1.5 times the Interquartile range to define my outliers.
Outliers can skew average values and misrepresent typical customer behavior. As a business owner one decision that can be affected by outliers is staffing you may expect more or less due to outliers and not use the right amount of staff. One that may not be affected is how you conduct marketing campaigns regardless of visits you will want to attract more customers.

In [ ]:
# Step 3: Check for missing values
missing_values_count = long_sample['dailyvisits'].isnull().sum()
missing_values_percentage = (missing_values_count / long_sample.shape[0]) * 100

print(f"Missing values count: {missing_values_count}")



Missing values count: 0


I don't have any missing values.

Question 3a (1 point): Aggregate your long sample to generate a table that summarizes the starting and ending dates of each year, # of observations in each year, and the mean of dailyvisits per year.

In [ ]:
# Step 1: Extract the year from the start date
long_sample['year'] = long_sample['date_range_start'].dt.year

# Step 2: Aggregate data by year
yearly_summary = long_sample.groupby('year').agg(
    start_date=('date_range_start', 'min'),
    end_date=('date_range_end', 'max'),
    num_observations=('dailyvisits', 'size'),
    mean_dailyvisits=('dailyvisits', 'mean')
).reset_index()

# Convert the mean_dailyvisits to two decimal places and format the dates
yearly_summary['mean_dailyvisits'] = yearly_summary['mean_dailyvisits'].round(2)
yearly_summary['start_date'] = yearly_summary['start_date'].dt.strftime('%Y-%m-%d')
yearly_summary['end_date'] = yearly_summary['end_date'].dt.strftime('%Y-%m-%d')

# Step 4: Display the summary table using tabulate
print(tabulate(yearly_summary, headers='keys', tablefmt='grid'))


+----+--------+--------------+------------+--------------------+--------------------+
|    |   year | start_date   | end_date   |   num_observations |   mean_dailyvisits |
+====+========+==============+============+====================+====================+
|  0 |   2018 | 2018-01-01   | 2019-01-07 |              79576 |              24.69 |
+----+--------+--------------+------------+--------------------+--------------------+
|  1 |   2019 | 2019-01-07   | 2020-01-06 |              78246 |              31.85 |
+----+--------+--------------+------------+--------------------+--------------------+
|  2 |   2020 | 2020-01-06   | 2021-01-04 |              78610 |              23.59 |
+----+--------+--------------+------------+--------------------+--------------------+
|  3 |   2021 | 2021-01-04   | 2022-01-03 |              78834 |              28.08 |
+----+--------+--------------+------------+--------------------+--------------------+
|  4 |   2022 | 2022-01-03   | 2023-01-02 |           

Question 3b (1 point): Define a new variable “dayofweek”. You can define the variable as numeric, with the value 1 for Monday, and …7 for Sunday; or as text with the values being “Monday”,..,”Sunday”.

(Hint: you need to create a date variable before this step. The date variable should give you the exact date of an observation. It can be created using the date_range_start variable and the variable you created when you reshape your data (the 'j' variable).

Aggregate dailyvisits by dayofweek in your long sample – that is, generate a table that tells you # of unique values in dayofweek, # of observations per value, and the mean of dailyvisits per value of dayofweek. On average, which day of week has the highest dailyvisits in your long sample?

In [ ]:
# Step 1: Create a date variable based on the start date and day number
long_sample['date'] = long_sample['date_range_start'] + pd.to_timedelta(
    long_sample['day'].str.extract('(\d+)')[0].astype(int) - 1, unit='d'
)

# Step 2: Define the dayofweek variable (1=Monday, ..., 7=Sunday)
long_sample['dayofweek'] = long_sample['date'].dt.dayofweek + 1

# Step 3: Aggregate dailyvisits by dayofweek
dayofweek_summary = long_sample.groupby('dayofweek').agg(
    num_observations=('dailyvisits', 'size'),
    mean_dailyvisits=('dailyvisits', 'mean')
).reset_index()

# Map numeric dayofweek to actual day names
day_names = {1: 'Monday', 2: 'Tuesday', 3: 'Wednesday', 4: 'Thursday', 5: 'Friday', 6: 'Saturday', 7: 'Sunday'}
dayofweek_summary['dayofweek_name'] = dayofweek_summary['dayofweek'].map(day_names)

# Format mean_dailyvisits to two decimal places
dayofweek_summary['mean_dailyvisits'] = dayofweek_summary['mean_dailyvisits'].round(2)

# Step 4: Determine which day has the highest average daily visits
highest_avg_day = dayofweek_summary.loc[dayofweek_summary['mean_dailyvisits'].idxmax()]

# Step 5: Display the summary table using tabulate
print(tabulate(dayofweek_summary[['dayofweek_name', 'num_observations', 'mean_dailyvisits']],
               headers=['Day of Week', 'Num Observations', 'Mean Daily Visits'],
               tablefmt='grid'))

# Step 6: Display the highest average daily visits
print(f"\nThe day with the highest average daily visits is {highest_avg_day['dayofweek_name']} "
      f"with an average of {highest_avg_day['mean_dailyvisits']:.2f} visits.")


+----+---------------+--------------------+---------------------+
|    | Day of Week   |   Num Observations |   Mean Daily Visits |
+====+===============+====================+=====================+
|  0 | Monday        |              56242 |               21.67 |
+----+---------------+--------------------+---------------------+
|  1 | Tuesday       |              56242 |               21.76 |
+----+---------------+--------------------+---------------------+
|  2 | Wednesday     |              56242 |               23.14 |
+----+---------------+--------------------+---------------------+
|  3 | Thursday      |              56242 |               24.53 |
+----+---------------+--------------------+---------------------+
|  4 | Friday        |              56242 |               34.38 |
+----+---------------+--------------------+---------------------+
|  5 | Saturday      |              56242 |               34.49 |
+----+---------------+--------------------+---------------------+
|  6 | Sun

Question 3c (2 point): Define a new variable “manyvisits” equal to 1 if dailyvisits is above a threshold and 0 otherwise. You can choose whatever threshold you like, it could be a constant or a function that depends on other variables in your data (e.g. the threshold may vary by season, weekend, or geography). Explain why you choose this threshold.

Tabulate the variable manyvisits and one geographic variable of your choice (e.g. the geographic variable can be state, zipcode, or a new variable of your own definition). By tabulation, I mean generating a table that tells you # of observations in each possible combination of your geographic variable and manyvisits. Which geographic unit has the highest # of observations with manyvisits=1? Which geographic unit has the highest percentage of observations with manyvisits=1? Do these findings make sense to you given the public information you can find online about your restaurant chain? (e.g. if you find that California has the highest # of observations with manyvisits=1 but the public information indicates that your chain never operates in California, then something must be wrong. Figure out why.)

In [ ]:
# Step 1: Calculate mean and standard deviation for weekdays (Monday to Thursday) and weekends (Friday to Sunday)
weekday_data = long_sample[long_sample['dayofweek'] <= 4]['dailyvisits']
weekend_data = long_sample[long_sample['dayofweek'] >= 5]['dailyvisits']

# Mean and standard deviation for weekdays
weekday_mean = weekday_data.mean()
weekday_std = weekday_data.std()

# Mean and standard deviation for weekends
weekend_mean = weekend_data.mean()
weekend_std = weekend_data.std()

# Step 2: Define thresholds as mean + 1 standard deviation
weekday_threshold = weekday_mean + weekday_std
weekend_threshold = weekend_mean + weekend_std

# Step 3: Create the manyvisits variable using the data-driven thresholds
long_sample['manyvisits'] = long_sample.apply(
    lambda row: 1 if (row['dailyvisits'] > weekend_threshold if row['dayofweek'] >= 5 else row['dailyvisits'] > weekday_threshold) else 0,
    axis=1
)

# Step 4: Choose a geographic variable (e.g., region)
geographic_var = 'region'

# Step 5: Tabulate the number of observations for each combination of manyvisits and the geographic variable
tabulation = long_sample.groupby([geographic_var, 'manyvisits']).size().unstack(fill_value=0)

# Add a column for total observations in each geographic unit
tabulation['total_observations'] = tabulation.sum(axis=1)

# Add a column for the percentage of manyvisits=1 in each geographic unit, rounded to 2 decimal places
tabulation['percent_manyvisits_1'] = ((tabulation[1] / tabulation['total_observations']) * 100).round(2)

# Step 6: Find the geographic unit with the highest number of manyvisits=1
highest_manyvisits_1 = tabulation[1].idxmax()
highest_manyvisits_1_count = tabulation[1].max()

# Step 7: Find the geographic unit with the highest percentage of manyvisits=1
highest_percent_manyvisits_1 = tabulation['percent_manyvisits_1'].idxmax()
highest_percent_manyvisits_1_value = tabulation['percent_manyvisits_1'].max()

# Step 8: Display the tabulation and results using tabulate
tabulated_data = tabulation.reset_index()
print(tabulate(tabulated_data, headers='keys', tablefmt='grid', showindex=False))

# Step 9: Display the highest number and percentage of manyvisits=1
print(f"\nThe geographic unit with the highest # of observations with manyvisits=1 is {highest_manyvisits_1} with {highest_manyvisits_1_count} observations.")
print(f"The geographic unit with the highest percentage of observations with manyvisits=1 is {highest_percent_manyvisits_1} with {highest_percent_manyvisits_1_value:.2f}% observations.")


+----------+-------+------+----------------------+------------------------+
| region   |     0 |    1 |   total_observations |   percent_manyvisits_1 |
+==========+=======+======+======================+========================+
| AL       |  1426 |  401 |                 1827 |                  21.95 |
+----------+-------+------+----------------------+------------------------+
| AR       |  2914 | 2567 |                 5481 |                  46.83 |
+----------+-------+------+----------------------+------------------------+
| AZ       | 16343 |  107 |                16450 |                   0.65 |
+----------+-------+------+----------------------+------------------------+
| CA       | 38449 |   23 |                38472 |                   0.06 |
+----------+-------+------+----------------------+------------------------+
| CO       |  8982 |   27 |                 9009 |                   0.3  |
+----------+-------+------+----------------------+------------------------+
| CT       |

In this analysis, I defined the variable "manyvisits" as 1 when daily visits exceeded a threshold calculated separately for weekdays (Monday to Thursday) and weekends (Friday to Sunday). The thresholds were derived from the mean and standard deviation of daily visits during each period.

To capture busier traffic patterns, the weekend threshold is set higher than the weekday threshold. Using one standard deviation allows us to define "manyvisits" as capturing the top 16% of daily visits, effectively identifying significant increases in customer traffic.

Question 3d (2 point): Define a new variable “core_biz_area” equal to 1 if the core business areas of your chain, and 0 otherwise. You can define core business areas whatever way you like, but please explain why you define it this way. (For example, you can define all states that have at least X units in your chain as the core business area and explain why you choose a specific number for X).

Aggregate the data by core_biz_area to show the # of observations and mean dailyvisits in core areas vs non-core areas. On average, do you observe more or fewer daily visits in the core areas? Could you name at least two reasons that may explain your finding?

In [ ]:
# Step 1: Calculate the threshold for core business areas (e.g., 75th percentile of observations by postal_code)
obs_per_postal_code = long_sample.groupby('postal_code')['dailyvisits'].size()
threshold = np.percentile(obs_per_postal_code, 75)

# Step 2: Define the core_biz_area variable (1 if postal_code has observations above the threshold, 0 otherwise)
long_sample['core_biz_area'] = long_sample['postal_code'].apply(
    lambda x: 1 if obs_per_postal_code[x] > threshold else 0
)

# Step 3: Aggregate data by core_biz_area
core_biz_summary = long_sample.groupby('core_biz_area').agg(
    num_observations=('dailyvisits', 'size'),
    mean_dailyvisits=('dailyvisits', 'mean')
).reset_index()

# Step 4: Display the aggregated data
print(tabulate(core_biz_summary, headers='keys', tablefmt='grid', showindex=False))

# Step 5: Interpret results (example reasoning)
core_area_visits = core_biz_summary[core_biz_summary['core_biz_area'] == 1]['mean_dailyvisits'].values[0]
non_core_area_visits = core_biz_summary[core_biz_summary['core_biz_area'] == 0]['mean_dailyvisits'].values[0]

print(f"Core areas have an average of {core_area_visits:.2f} daily visits compared to {non_core_area_visits:.2f} in non-core areas.")


+-----------------+--------------------+--------------------+
|   core_biz_area |   num_observations |   mean_dailyvisits |
+=================+====================+====================+
|               0 |             351512 |            27.3127 |
+-----------------+--------------------+--------------------+
|               1 |              42182 |            23.0319 |
+-----------------+--------------------+--------------------+
Core areas have an average of 23.03 daily visits compared to 27.31 in non-core areas.


**Explaining Process for Core area**
To find my core business areas I used the top 25 percent of my observations grouped on the data by postal_code. By doing this I track the number of daily visits for each postal code. The result is a series where the postal codes are the index, and the value is the number of observations for that postal code and I filtered out the top quartile as my core business areas.

**Explanation of daily visits**
On average, non-core areas have more daily visits (27.31) than core areas (23.03). This could be due to less competition in non-core areas, where fewer nearby options lead to higher traffic per outlet. Additionally, core areas often have more outlets concentrated in urban regions, causing customer traffic to be spread across multiple locations, reducing the daily visits per individual restaurant.

In [ ]:
long_sample.to_csv('/content/drive/MyDrive/ECON_433_Project_2_data/long_sample.csv', index=False)
